In [1]:
from __future__ import print_function
import ipywidgets as ipw
from IPython.display import display
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE) 
from aiidalab_widgets_base import StructureUploadWidget, CodeDropdown
%matplotlib notebook

layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

## Step 1: Select Structure

Before uploading, please
 * **Desolvate** the structure
 * Use **P1 symmetry**

In [ ]:
widget = StructureUploadWidget(node_class='CifData')
display(widget)

## Step 2: Select executable

In [ ]:
dropdown = CodeDropdown(input_plugin='zeopp.network')
display(dropdown)

## Step 3: Compute 

In [ ]:
import ipywidgets as ipw
import pandas as pd
from aiida_zeopp.workflows import ZeoppBlockPocketsWorkChain
from aiida.orm.data.base import Float
from aiida.work.run import run
from IPython.display import FileLink

probe_radius = ipw.BoundedFloatText(
    description='Probe radius [Angstrom]:', 
    min=0.1,
    max=100.0,
    value=1.8,
    layout=layout,
    style=style
)


results = None
def on_click_submit(b):
    global results
    structure = widget.structure_node

    if structure is None:
        message.value = "Please upload a structure first"
        return None
        
    code = dropdown.selected_code
    if code is None:
        message.value = "Please select a code first"
        return None

    message.value = 'Please wait, pore properties are being computed (can take ~10 minutes).'
    btn_submit.disabled = True

    with submit_out:
        results = run(
            ZeoppBlockPocketsWorkChain,
            probe_radius=Float(probe_radius.value),
            structure=structure,
            zeopp_code=code
        )
        
    plot(results)
        

def plot(results):
    
    cif_filename = widget.structure_node.filename
    
    d = results['output_parameters'].get_dict()
    
    # Hide input data
    for k in list(d.keys()):
        if k.startswith('Input_'):
            d.pop(k, None)

    df = pd.DataFrame.from_dict(d, orient="index", columns=[cif_filename])
    
    message.value = df.to_html()
    
    results_csv = '{}.csv'.format(cif_filename)
    df.to_csv(results_csv)
    display(FileLink(results_csv, result_html_prefix='Download '))


btn_submit = ipw.Button(description='Compute Properties')
btn_submit.on_click(on_click_submit)
message = ipw.HTML('', layout=layout)
submit_out = ipw.Output()

display(
    probe_radius,
    btn_submit,
    submit_out,
    message
)